In [1]:
import sys
import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

def date_time(table_cells):
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    out=[i for i in table_cells.strings][0]
    return out

def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass

def extract_column_from_header(row):
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
    colunm_name = ' '.join(row.contents)
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name

In [2]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"
headers = {"User-Agent": "Mozilla/5.0"}

# TASK 1: Request the page and create BeautifulSoup object
response = requests.get(static_url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')

# Verify by printing title
print(soup.title.string)

List of Falcon 9 and Falcon Heavy launches - Wikipedia


In [3]:
# TASK 2: Extract column names from the HTML table header
html_tables = soup.find_all('table')
first_launch_table = html_tables[2] # Target table
column_names = []

temp = first_launch_table.find_all('th')
for row in temp:
    name = extract_column_from_header(row)
    if name is not None and len(name) > 0:
        column_names.append(name)

print("Extracted Columns:", column_names)

Extracted Columns: ['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome']


In [4]:
launch_dict = dict.fromkeys(column_names)

# Remove irrelevant columns and initialize empty lists
if 'Date and time ( )' in launch_dict:
    del launch_dict['Date and time ( )']

launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []
launch_dict['Version Booster'] = []
launch_dict['Booster landing'] = []
launch_dict['Date'] = []
launch_dict['Time'] = []

In [5]:
# TASK 3: Parse the launch HTML tables
extracted_row = 0
for table_number, table in enumerate(soup.find_all('table', "wikitable plainrowheaders collapsible")):
    for rows in table.find_all("tr"):
        if rows.th:
            if rows.th.string:
                flight_number = rows.th.string.strip()
                flag = flight_number.isdigit()
        else:
            flag = False
        
        row = rows.find_all('td')
        if flag:
            extracted_row += 1
            launch_dict['Flight No.'].append(flight_number)
            
            # Date and Time
            datatimelist = date_time(row[0])
            launch_dict['Date'].append(datatimelist[0].strip(','))
            launch_dict['Time'].append(datatimelist[1])
              
            # Booster version
            bv = booster_version(row[1])
            if not(bv) and row[1].a:
                bv = row[1].a.string
            launch_dict['Version Booster'].append(bv)
            
            # Launch Site, Payload, Orbit
            launch_dict['Launch site'].append(row[2].a.string if row[2].a else None)
            launch_dict['Payload'].append(row[3].a.string if row[3].a else None)
            launch_dict['Payload mass'].append(get_mass(row[4]))
            launch_dict['Orbit'].append(row[5].a.string if row[5].a else None)
            
            # Customer
            if row[6].a:
                launch_dict['Customer'].append(row[6].a.string)
            else:
                launch_dict['Customer'].append(row[6].string.strip() if row[6].string else None)
            
            # Outcome and Landing
            launch_dict['Launch outcome'].append(list(row[7].strings)[0] if row[7].strings else None)
            launch_dict['Booster landing'].append(landing_status(row[8]))

# Create DataFrame
df = pd.DataFrame({ key:pd.Series(value) for key, value in launch_dict.items() })

In [9]:
df.to_csv('spacex_web_scraped.csv', index=False)
print(df.head())

  Flight No. Launch site                               Payload Payload mass  \
0          1       CCAFS  Dragon Spacecraft Qualification Unit            0   
1          2       CCAFS                                Dragon            0   
2          3       CCAFS                                Dragon       525 kg   
3          4       CCAFS                          SpaceX CRS-1     4,700 kg   
4          5       CCAFS                          SpaceX CRS-2     4,877 kg   

  Orbit Customer Launch outcome   Version Booster Booster landing  \
0   LEO   SpaceX      Success\n  F9 v1.07B0003.18         Failure   
1   LEO     NASA        Success  F9 v1.07B0004.18         Failure   
2   LEO     NASA        Success  F9 v1.07B0005.18    No attempt\n   
3   LEO     NASA      Success\n  F9 v1.07B0006.18      No attempt   
4   LEO     NASA      Success\n  F9 v1.07B0007.18    No attempt\n   

              Date   Time  
0      4 June 2010  18:45  
1  8 December 2010  15:43  
2      22 May 2012  07:44 